In [146]:
from flask import Flask, Response
from flask_caching import Cache
import uuid
import random
import collections
import numpy as np
import json


In [107]:
class Game:
    status = 'waiting' # waiting, started, end
    gameid = ''
    members = {}
    routelist = []
    routeidx = 0
    cards = []
    stocks = list(range(2, 99))
    hightolow = []
    lowtohigh = []


class Member:
    nickname = ''
    holdcards = []


In [108]:
print(Game.stocks)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98]


In [109]:
# create the game
game = Game()
member = Member()

gameid = str(uuid.uuid4())
game.gameid = gameid
member.nickname = gameid
game.members[gameid] = member
print(gameid)


5d6ebd99-60de-40be-8e0f-e12d0b8bdd0e


In [111]:
# join member
member = Member()

clientid = str(uuid.uuid4())
member.nickname = clientid
member.holdcards = []
game.members[clientid] = member
print(clientid)

b6b24d41-1bb2-48d4-9115-b97233767efc


In [112]:
for mid in game.members.keys():
    print(mid)

5d6ebd99-60de-40be-8e0f-e12d0b8bdd0e
edbf43e8-a0e4-4ec3-b3dd-340fa10ec7d6
b6b24d41-1bb2-48d4-9115-b97233767efc


In [113]:
# start the game
members = [mid for mid in game.members.keys()]
random.shuffle(members)
game.routelist = members

for mid in members:
    member = game.members[mid]
    member.holdcards = []
    for i in range(6):
        member.holdcards.append(game.stocks.pop(random.randint(0, len(game.stocks) - 1)))

game.hightolow = []
game.lowtohigh = []
game.hightolow.append([100])
game.hightolow.append([100])
game.lowtohigh.append([1])
game.lowtohigh.append([1])


In [143]:
print(game.routelist)

for mid in game.members.keys():
    print(game.members[mid].holdcards, mid)

print('[0]', game.hightolow[0][-1])
print('[1]', game.hightolow[1][-1])
print('[2]', game.lowtohigh[0][-1])
print('[3]', game.lowtohigh[1][-1])


['edbf43e8-a0e4-4ec3-b3dd-340fa10ec7d6', 'b6b24d41-1bb2-48d4-9115-b97233767efc', '5d6ebd99-60de-40be-8e0f-e12d0b8bdd0e']
[57, 18, 59, 88, 83, 97] 5d6ebd99-60de-40be-8e0f-e12d0b8bdd0e
[31, 51, 15, 8, 35, 37] edbf43e8-a0e4-4ec3-b3dd-340fa10ec7d6
[71, 10, 32, 16, 12, 20] b6b24d41-1bb2-48d4-9115-b97233767efc
[0] 93
[1] 70
[2] 65
[3] 1


In [140]:
# use card
lineid = 0
clientid = 'edbf43e8-a0e4-4ec3-b3dd-340fa10ec7d6'
cardnum = 93

if lineid in [0, 1]:
    # 100 -> 2
    if game.hightolow[lineid][-1] > cardnum:
        game.hightolow[lineid].append(cardnum)
    else:
        if (game.hightolow[lineid][-1] + 10) == cardnum:
            game.hightolow[lineid].append(cardnum)
        else:
            print('Error1')
elif lineid in [2, 3]:
    # 1 -> 99
    if game.lowtohigh[lineid%2][-1] < cardnum:
        game.lowtohigh[lineid%2].append(cardnum)
    else:
        if (game.lowtohigh[lineid%2][-1] - 10) == cardnum:
            game.lowtohigh[lineid%2].append(cardnum)
        else:
            print('Error3')
else:
    print('Error5')

game.members[clientid].holdcards.remove(cardnum)


In [142]:
game.routeidx = (game.routeidx + 1) % 6

for mid in members:
    member = game.members[mid]
    while len(member.holdcards) < 6:
        member.holdcards.append(game.stocks.pop(random.randint(0, len(game.stocks) - 1)))


In [78]:
print(game.routeidx)
for mid in game.members.keys():
    print(game.members[mid].holdcards, mid)

1
[62, 78, 48, 79, 22, 95] 15bc6063-b3c3-4df4-8e88-cacec2246e76
[57, 46, 49, 5, 20, 71] dd0e0b41-8457-42aa-a4bd-1c8fe549d5b1
[86, 94, 90, 53, 42, 6] e9812337-bbf8-4a5a-9d6f-d40631e5847d


In [101]:
yourturn = False

if game.routelist[game.routeidx] == clientid:
    yourturn = True

holdcards = game.members[clientid].holdcards
print(yourturn, holdcards)

True [65, 40, 23, 46, 94]


In [147]:
    response = {'turn': yourturn, 'holdcards': holdcards}
json.dumps(response)

'{"turn": true, "holdcards": [65, 40, 23, 46, 94]}'